In [1]:
from konlpy.tag import Komoran
from konlpy.tag import Kkma
import pandas as pd
import numpy as np
from re import match
from gensim.models.word2vec import Word2Vec

In [51]:
#data = pd.read_csv("./crawling_result_daum.csv")
data = pd.read_csv("./crawling_result_naver.csv", 
                 names=['제목','작가','평점','장르','연령','한줄 스토리'])

story = data[['한줄 스토리']]

print(len(story))


1199


In [3]:
komoran = Komoran()
kkma = Kkma()

keywords = []
temp = []
empty_list = [' ']

# kkma와 komoran 두클래스에서 모두 일반명사이고, 길이가 2이상인 단어들
for i in story.index:
    try:
        val = story._get_value(i,"한줄 스토리")
        words = komoran.pos(val)
        for(text, tclass) in words:
            if tclass=='NNG' and not len(text)==1 and not(match('^[0-9]',text)):
                temp.append(text)
        words = kkma.pos(val)
        for(text, tclass) in words:
            if tclass=='NNG' and not len(text)==1 and not(match('^[0-9]',text)):
                temp.append(text)
        keywords.append(temp.copy())
        temp.clear()
        if (i%10)==0:
            print(i,' ', end='')
    except:
        keywords.append(empty_list.copy())

#중복제거
newkeywords = []
for i in range(len(keywords)):
    newkeywords.append(list(set(keywords[i])))

0  10  20  30  40  50  60  70  80  90  100  110  120  130  140  150  160  170  180  190  200  210  220  230  240  250  260  270  280  290  300  310  320  330  340  350  360  370  380  390  400  410  420  430  440  450  460  470  480  490  500  510  520  530  540  550  560  570  580  590  600  610  620  630  640  650  660  670  680  690  700  710  720  730  740  750  760  770  780  790  800  810  820  830  840  850  860  870  880  890  900  910  920  930  940  950  960  970  980  990  1000  1010  1020  1030  1040  1050  1060  1070  1080  1090  1100  1110  1120  1130  1140  1150  1160  1170  1180  1190  1200  

In [4]:
# 삭제할 특정 단어 리스트
delete_words = ['이의', '작품', '만화','남자','여자', '자신', '스스로', 
                '의의' ,'사이','남성','여성','만이','작가', '인간', '사람', 
                '시작', '신작', '이번','과의','순간','이야기','내가', '지금',
                '다운']
for i in range(len(newkeywords)):
    for v in newkeywords[i]:
        if v in delete_words:
            newkeywords[i].remove(v)

for i in range(len(newkeywords)):
    for v in newkeywords[i]:
        if v in delete_words:
            newkeywords[i].remove(v)

In [5]:
# 학원물
school_words = ['학교','학원','전교','교실','교내','중학생','고등학생','여고생','여학생','남학생','전학생','전학','고교']
isSchool = []
next_webtoon = False
for i in range(len(newkeywords)):
    for v in newkeywords[i]:
        if v in school_words:
            isSchool.append(1)
            next_webtoon = True
            break
    if next_webtoon:
        next_webtoon = False
        continue
    else:
        isSchool.append(0)

In [6]:
#for i in range(len(newkeywords)):
#    print(i,': ', end='')
#    print(newkeywords[i])

In [48]:
#data['스토리 키워드'] = newkeywords
#data['학원물'] = isSchool

integration_data = pd.read_csv("./naver_daum_integration_v2.csv")
integration_data['keywords'] = newkeywords

ValueError: Length of values (1202) does not match length of index (2383)

In [8]:
# data.to_csv("./storykeywords_result_daum.csv", header=True, index = False)
# data.to_csv("./storykeywords_result_naver.csv", header=True, index = False)

In [44]:
# ['대학생', '수학', '일상', '쓰레기'] 형태로 단어만 학습
# 참고 블로그 : http://blog.daum.net/geoscience/1414
keywords_model = Word2Vec(newkeywords, sg=1,window=4, min_count=1,iter=10)
keywords_model.init_sims(replace=True)

print("유사도 : %.5f "%(keywords_model.wv.similarity('동물','성공')))

print("< 가장 유사한 단어 10개 >")
print(keywords_model.wv.most_similar("로맨스",topn=10))

유사도 : 0.53156 
< 가장 유사한 단어 10개 >
[('사건', 0.8879833221435547), ('소녀', 0.8827046155929565), ('액션', 0.8722118139266968), ('사랑', 0.862705647945404), ('비밀', 0.858494758605957), ('친구', 0.8542833924293518), ('청춘', 0.8491671085357666), ('드라마', 0.844719409942627), ('복수', 0.8430224657058716), ('세계', 0.8411802053451538)]


In [10]:
genre_list = data['장르'].values.tolist()
keywords_list = data['스토리 키워드'].values.tolist()

genre_and_keywords = []
temp =[]
for i in range(len(keywords_list)):
    temp_genre = genre_list[i]
    temp_genre = temp_genre.replace("'","")
    temp_genre = temp_genre.replace("[","")
    temp_genre = temp_genre.replace("]","")
    temp_genre = temp_genre.replace(" ","")
    for word in temp_genre.split(','):
        temp.append(word)
    for word in keywords_list[i]:
        temp.append(word)
    genre_and_keywords.append(temp.copy())
    temp.clear()
    

In [30]:
# 장르 + 키워드 모델
genre_keywords_model = Word2Vec(genre_and_keywords, sg=1,window=4, min_count=1, iter=10)
genre_keywords_model.init_sims(replace=True)

#print("유사도 : %.5f "%(genre_keywords_model.wv.similarity('로맨스','공포')))

#print("< 가장 유사한 단어 10개 >")
#print(genre_keywords_model.wv.most_similar("로맨스",topn=200))
result = genre_keywords_model.wv.most_similar("로맨스",topn=10)
print(result)

[('사건', 0.9994421005249023), ('복수', 0.9993911981582642), ('코믹', 0.9993472099304199), ('친구', 0.999339759349823), ('생활', 0.9993105530738831), ('액션', 0.999300479888916), ('스릴러', 0.9992736577987671), ('청춘', 0.9992637634277344), ('인생', 0.9992629885673523), ('좌충우돌', 0.9992598295211792)]


In [12]:
# 장르제외한 키워드와 유사한 키워드들
input_keywords = newkeywords[0]
print(input_keywords)
similar_words = []
for word in input_keywords:
    for item in genre_keywords_model.wv.most_similar(word,topn=30):
        similar_words.append(item)
        
for word in similar_words:
    for other in similar_words:
        if word[0]==other[0] and word[1]!=other[1]:
            similar_words.remove(other)
similar_words.sort(key=lambda x:x[1], reverse = True)
print(similar_words[0:50])


['동물', '한철', '성공', '전부', '유명', '수의사', '소리']
[('외모', 0.999409556388855), ('인터넷', 0.9994074106216431), ('저택', 0.9993913173675537), ('규칙적', 0.999385416507721), ('마애', 0.999373197555542), ('자신감', 0.9993664622306824), ('품행', 0.9993640184402466), ('카툰', 0.9993570446968079), ('취직', 0.9993482828140259), ('병이', 0.9993380904197693), ('라디오', 0.9993370175361633), ('사파', 0.9993332028388977), ('조카', 0.999331533908844), ('삼존', 0.9993294477462769), ('멀티플렉스', 0.9993266463279724), ('난쟁이', 0.9993235468864441), ('리처드', 0.9993202090263367), ('극도', 0.9993196725845337), ('컬링', 0.9993194341659546), ('신흥', 0.9993168115615845), ('여래', 0.9993159174919128), ('후인', 0.9993144273757935), ('행운', 0.9993119239807129), ('은유', 0.9993119239807129), ('교사', 0.9993115663528442), ('그룹', 0.9993101358413696), ('차이', 0.9993101358413696), ('인터뷰', 0.9993009567260742), ('효자손', 0.9992988705635071), ('악당', 0.9992791414260864), ('접속', 0.999276876449585), ('레이션', 0.9992711544036865), ('강주', 0.999264121055603), ('생과', 0.9992541670799255

In [45]:
input_keywords = newkeywords[0]
print(input_keywords)
#keywords_model.wv.similarity('로맨스','성당')
#compare_keywords = newkeywords[200]
#print(compare_keywords)
similarity_score_result = []
for compare_keywords in newkeywords:
    sum=0
    for compare_word in compare_keywords:
        for input_word in input_keywords:
            similarity = keywords_model.wv.similarity(compare_word,input_word)
            #print(compare_word," and ",input_word," : ",similarity)
            sum+=similarity
    try:
        avg=sum/(len(compare_keywords)*len(input_keywords))
        similarity_score_result.append(avg)
    except:
        similarity_score_result.append(empty_list.copy())

print(len(similarity_score_result))

['동물', '한철', '성공', '전부', '유명', '수의사', '소리']
1202


In [13]:
# 문장으로 학습하기..?
story_list = story.values.tolist()
#story_model = Word2Vec(story_list, sg=1,window=4, min_count=1)
#story_model.init_sims(replace=True)
# print(list(story_model.wv.vocab.keys()))
print(story_list[0][0])

돈과 성공만을 전부로 알던 유명 수의사 한철은, 어느 날 동물들의 소리를 들을 수 있게 되는데...


In [14]:
# soynlp

from soynlp.tokenizer import LTokenizer
from soynlp.word import WordExtractor
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor
from soynlp.tokenizer import MaxScoreTokenizer

In [15]:
tokenizer = LTokenizer()
# tokenizer = MaxScoreTokenizer()
#print(tokenizer.tokenize(story_list[1][0]))
allstory=""
for i in range(len(story_list)):
    allstory = allstory + " " + story_list[i][0]

print(allstory)

 돈과 성공만을 전부로 알던 유명 수의사 한철은, 어느 날 동물들의 소리를 들을 수 있게 되는데... 자신의 운명을 스스로 개척하기 위해 궁을 벗어난 공주님의 좌충우돌 모험 로맨스 '해와 달' 속 달님, 천방지축 선녀와 지상 최대 싸가지 남자가 펼치는 판타지 로맨스! 대학생 시영이와 외로운 뱀파이어의 심쿵 먹방 동거생활! 스트리머 현세이의 일상 소통 이야기 칼퇴를 꿈꾸는 속물 표사 무영! 하지만 사부님의 희망사항은? '협객이 되어라!' 스무 살, 이루지 못했던 첫사랑과 스물아홉에 다시 만났다. 이번엔 그 때와 다를 수 있을까? 햇살 같은 하나와 음침 미소년 창욱이의 친구 만들기 프로젝트! 다소 쓰레기 같은 대학생 작가의 일상을 다룬 만화. 수학을 싫어합니다. 탐욕을 향한 호구들의 몸부림, 그리고 펼쳐지는 대 주식사기극 너무 다른 두 사람의 연애 3년, 민성의 비밀은 두 사람을 예기치 못 한 사건에 휘말리게 한다. 내가 가르치는 보컬수업에, 우리 과 교수님이 수강생으로 들어왔다. 조류공포증이 있는 철민이 돌연변이 새들과 하르퓌아로부터 가족을 지키는 이야기  사상 최악의 막장 쓰레기 문제아 학교, 정문여상에서 살아남기! 엑소시스트가 되고자 하는 소녀의 이야기 SF액션 활극, 쑈오리 라마가 세상을 바꾼다!  소소한 생활지름의 세계로 오라~  트레이스, 트러블, 인간.. 그들의 생존을 건 싸움 한끼 한끼 최선을 다 해 먹는 조경규 작가의 맛나는 세상! 저마다의 사정으로 피 튀는 싸움판에 뛰어든 소년들. 갖고 싶다면 뺏어라! 사라진(?) 신랑을 찾기 위한 톱스타 진사랑의 고군분투 사랑이야기 의미 없는 삶에 지친 고3 영진, 비밀이 많아 보이는  전학생 유은과의 만남으로 변화가 생긴 삶 유명 인터넷 방송인과 그 편집자를 둘러싼 끔찍한 살인사건... 과연 진실은? 유도선수를 그만두고 졸업을 기다리던 황희는 PD지망생 요한을 만나 방송을 찍게 된다! 심장 이식 수술을 받은 소유. 이식받은 심장이 같은 반 남자애 아빠의 심장이라고? 미웠던 사람이 좋아지는 건 한 순간! 두근두

In [16]:
word_extractor = WordExtractor()
word_extractor.train(allstory)
words = word_extractor.extract()
print(words)

training was done. used memory 0.906 Gb
all cohesion probabilities was computed. # words = 0
all branching entropies was computed # words = 0
all accessor variety was computed # words = 0
{}
